### Introduction
Going from the previous version 'omnidrone_alternative_dynamics.ipynb', I realised that the problem was gimbal lock: The $q_{2}=0$ condition corresponds to the pitch (i.e. the rotation about y) of the EE frame in the zero frame to be $\pi/2$, yielding a singularity. To fix this, we redefine the zero frame and go through the derivation again.

In [65]:
import sympy as sp
import numpy as np

sp.init_printing(use_latex=True)

L1_length = 0.110
L2_length = 0.311
L3_length = 0.273

#q1, q2, q3 = sp.symbols('q_1 q_2 q_3', real=True)
x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3 = sp.symbols('x_b y_b z_b yaw pitch roll q_1 q_2 q_3', real=True)
state_vector = sp.Matrix([x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3])

# Secondary states
x_be, y_be, z_be = sp.symbols('x^{b}_{be} y^{b}_{be} z^{b}_{be}')

# Parameters
L1, L2, L3 = sp.symbols('L_1 L_2 L_3', real=True, positive=True)

def Rx(x):
    return sp.Matrix([[1, 0, 0], [0, sp.cos(x), -sp.sin(x)], [0, sp.sin(x), sp.cos(x)]])

def Ry(y):
    return sp.Matrix([[sp.cos(y), 0, sp.sin(y)], [0, 1, 0], [-sp.sin(y), 0, sp.cos(y)]])

def Rz(z):
    return sp.Matrix([[sp.cos(z), -sp.sin(z), 0], [sp.sin(z), sp.cos(z), 0], [0, 0, 1]])

In [66]:
Rz_yaw = Rz(yaw)

Ry_pitch = Ry(pitch)

Rx_roll = Rx(roll)

# Intrinsic ZYX: Postmultiplication
R_Ib = Rz_yaw * Ry_pitch * Rx_roll
sp.pprint(R_Ib)

T_euler = sp.Matrix([[0, -sp.sin(yaw), sp.cos(yaw)*sp.cos(pitch)],
                     [0, sp.cos(yaw), sp.sin(yaw)*sp.cos(pitch)],
                     [1, 0, -sp.sin(pitch)]])

T_A = sp.Matrix([[sp.eye(3), sp.zeros(3, 3)],
                 [sp.zeros(3, 3), T_euler]])

H_Ib = sp.Matrix([[sp.cos(yaw)*sp.cos(pitch), sp.cos(yaw)*sp.sin(pitch)*sp.sin(roll)-sp.sin(yaw)*sp.cos(roll), sp.cos(yaw)*sp.sin(pitch)*sp.cos(roll)+sp.sin(yaw)*sp.sin(roll), x_b],
                  [sp.sin(yaw)*sp.cos(pitch), sp.sin(yaw)*sp.sin(pitch)*sp.sin(roll)+sp.cos(yaw)*sp.cos(roll), sp.sin(yaw)*sp.sin(pitch)*sp.cos(roll)-sp.cos(yaw)*sp.sin(roll), y_b],
                  [-sp.sin(pitch), sp.cos(pitch)*sp.sin(roll), sp.cos(pitch)*sp.cos(roll), z_b],
                  [0, 0, 0, 1]])

⎡cos(pitch)⋅cos(yaw)  sin(pitch)⋅sin(roll)⋅cos(yaw) - sin(yaw)⋅cos(roll)  sin( ↪
⎢                                                                              ↪
⎢sin(yaw)⋅cos(pitch)  sin(pitch)⋅sin(roll)⋅sin(yaw) + cos(roll)⋅cos(yaw)  sin( ↪
⎢                                                                              ↪
⎣    -sin(pitch)                     sin(roll)⋅cos(pitch)                      ↪

↪ pitch)⋅cos(roll)⋅cos(yaw) + sin(roll)⋅sin(yaw)⎤
↪                                               ⎥
↪ pitch)⋅sin(yaw)⋅cos(roll) - sin(roll)⋅cos(yaw)⎥
↪                                               ⎥
↪            cos(pitch)⋅cos(roll)               ⎦


In [67]:
# Denavit-Hartenberg approach
# Parameters
DH_theta = [q1, q2, q3]
DH_alpha = [-sp.pi/2, sp.pi/2, 0]
DH_r = [L1, L2, L3]
DH_d = [0, 0, 0]

H_b0 = sp.Matrix([[ 0, 0, 1, 0],
                  [ 0, 1, 0, 0],
                  [-1, 0, 0, 0],
                  [ 0, 0, 0, 1]])

# Frame 0: X pointing right, Z pointing forward, Y pointing down (i.e. rotation around Zb by pi/2, then rotation around Xb' by pi/2)
# I.e. intrinisic rotation so multiply Rz * Rx'
# Frame 1: Attached at Q2, Z pointing down, X pointing right (rotation around X0 of -pi/2, then translation along x0' by L1)

H_01 = sp.Matrix([[sp.cos(DH_theta[0]), -sp.sin(DH_theta[0])*sp.cos(DH_alpha[0]), sp.sin(DH_theta[0])*sp.sin(DH_alpha[0]), DH_r[0]*sp.cos(DH_theta[0])],
                  [sp.sin(DH_theta[0]), sp.cos(DH_theta[0])*sp.cos(DH_alpha[0]), -sp.cos(DH_theta[0])*sp.sin(DH_alpha[0]), DH_r[0]*sp.sin(DH_theta[0])],
                  [0, sp.sin(DH_alpha[0]), sp.cos(DH_alpha[0]), DH_d[0]],
                  [0, 0, 0, 1]])

# Frame 2: Attached at Q3, Z pointing forward, X pointing right (rotation around X1 of pi/2, then translation along x1' by L2)
H_12 = sp.Matrix([[sp.cos(DH_theta[1]), -sp.sin(DH_theta[1])*sp.cos(DH_alpha[1]), sp.sin(DH_theta[1])*sp.sin(DH_alpha[1]), DH_r[1]*sp.cos(DH_theta[1])],
                  [sp.sin(DH_theta[1]), sp.cos(DH_theta[1])*sp.cos(DH_alpha[1]), -sp.cos(DH_theta[1])*sp.sin(DH_alpha[1]), DH_r[1]*sp.sin(DH_theta[1])],
                  [0, sp.sin(DH_alpha[1]), sp.cos(DH_alpha[1]), DH_d[1]],
                  [0, 0, 0, 1]])

# Frame 3: Attached at EE, Z pointing forward, X pointing right (translation along x2 by L3)
H_23 = sp.Matrix([[sp.cos(DH_theta[2]), -sp.sin(DH_theta[2])*sp.cos(DH_alpha[2]), sp.sin(DH_theta[2])*sp.sin(DH_alpha[2]), DH_r[2]*sp.cos(DH_theta[2])],
                  [sp.sin(DH_theta[2]), sp.cos(DH_theta[2])*sp.cos(DH_alpha[2]), -sp.cos(DH_theta[2])*sp.sin(DH_alpha[2]), DH_r[2]*sp.sin(DH_theta[2])],
                  [0, sp.sin(DH_alpha[2]), sp.cos(DH_alpha[2]), DH_d[2]],
                  [0, 0, 0, 1]])

# Transformation between 3 and e - To align e with frame e from direct derivation
H_3e = sp.Matrix([[ 0,  0, 1, 0],
                  [ 0,  1, 0, 0],
                  [-1,  0, 0, 0],
                  [ 0,  0, 0, 1]])

# Equivalent transformation
H_be = (H_b0 * H_01 * H_12 * H_23 * H_3e).subs([(L1, L1_length), (L2, L2_length), (L3, L3_length)])

In [68]:
# Testing if the DH formulation yielded the same result as the direct derivation
# Unit test 1
q1_val = 0.0
q2_val = 0
q3_val = 0
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)]).evalf()
if (H_be_subs[2,3] + (L1_length+L2_length+L3_length)< 0.0001):
    print("Unit test 1 passed")
else:
    print("Unit test 1 failed")

# Unit test 2
q1_val = sp.pi/2
q2_val = 0
q3_val = 0
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)]).evalf()
if (H_be_subs[1,3] - (L1_length+L2_length+L3_length)< 0.0001):
    print("Unit test 2 passed")
else:
    print("Unit test 2 failed")

# Unit test 3
q1_val = sp.pi/2-sp.pi/6
q2_val = 0.0
q3_val = sp.pi/6
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)]).evalf()
if ((H_be_subs[1,3] - (sp.cos(sp.pi/6).evalf()*(L1_length+L2_length)+L3_length)< 0.0001) and (H_be_subs[2,3] + (sp.sin(sp.pi/6).evalf()*(L1_length+L2_length))< 0.0001)):
    print("Unit test 3 passed")
else:
    print("Unit test 3 failed")

# Unit test 4
q1_val = sp.pi/2
q2_val = sp.pi/6
q3_val = 0.0
H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)]).evalf()
if ((H_be_subs[0,3] + (sp.sin(sp.pi/6).evalf()*(L3_length+L2_length))< 0.0001) and (H_be_subs[1,3] - (sp.cos(sp.pi/6).evalf()*(L3_length+L2_length)+L1_length)< 0.0001)):
    print("Unit test 4 passed")
else:
    print("Unit test 4 failed")

Unit test 1 passed
Unit test 2 passed
Unit test 3 passed
Unit test 4 passed


In [69]:
# Forward kinematics function of the manipulator (in the body frame)
FK_manipulator = sp.Matrix([H_be[0,3],
                H_be[1,3],
                H_be[2,3],
                sp.atan2(H_be[1,0], H_be[0,0]), # yaw
                -sp.asin(H_be[2,0]), # pitch
                sp.atan2(H_be[2,1], H_be[2,2])]) # roll

FK_manipulator = sp.simplify(sp.trigsimp(FK_manipulator))

q1_val = sp.pi/2-sp.pi/6
q2_val = 0.0
q3_val = sp.pi/6
FK_manipulator_subs = FK_manipulator.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val)]).evalf()
print("Forward kinematics of the manipulator (in the body frame)")
sp.pprint(FK_manipulator_subs)

Forward kinematics of the manipulator (in the body frame)
⎡        0        ⎤
⎢                 ⎥
⎢0.637596694993249⎥
⎢                 ⎥
⎢     -0.2105     ⎥
⎢                 ⎥
⎢3.14159265358979 ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎣ 1.5707963267949 ⎦


In [70]:
# Forward kinematics function of the combined kinematics (in the inertial frame)
H_Ie = H_Ib * H_be
FK_centralised = sp.Matrix([H_Ie[0,3],
                H_Ie[1,3],
                H_Ie[2,3],
                sp.atan2(H_Ie[1,0], H_Ie[0,0]), # yaw
                -sp.asin(H_Ie[2,0]), # pitch
                sp.atan2(H_Ie[2,1], H_Ie[2,2])]) # roll
FK_centralised = sp.trigsimp(sp.expand(FK_centralised))

sp.pprint("State dependencies of centralised kinematics")
print(f"xe = f({FK_centralised[0,0].free_symbols})")
print(f"ye = f({FK_centralised[1,0].free_symbols})")
print(f"ze = f({FK_centralised[2,0].free_symbols})")
print(f"yawe = f({FK_centralised[3,0].free_symbols})")
print(f"pitche = f({FK_centralised[4,0].free_symbols})")
print(f"rolle = f({FK_centralised[5,0].free_symbols})")

sp.pprint("Denominators for yaw and roll")
sp.pprint(H_be[0,0])
sp.pprint(H_be[2,2])

# Jacobian calculation for velocity
J_A_centralised = FK_centralised.jacobian([x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3])

print(f"Centralised Jacobian shape: {J_A_centralised.shape}")

# Instantaneous states
xb_inst = 0.0
yb_inst = 0.0
zb_inst = 0.0
yaw_inst = 0.0
pitch_inst = 0.0
roll_inst = 0.0
q1_inst = sp.pi/2-sp.pi/6
q2_inst = 0
q3_inst = sp.pi/6

# Substitute the instantaneous states
J_A_centralised_inst = J_A_centralised.subs([(x_b, xb_inst), (y_b, yb_inst), (z_b, zb_inst), (yaw, yaw_inst), (pitch, pitch_inst), (roll, roll_inst), (q1, q1_inst), (q2, q2_inst), (q3, q3_inst)]).evalf()
sp.pprint(J_A_centralised_inst)
print(J_A_centralised_inst.rank())

# Evaluate the FK at this state
FK_centralised_inst = FK_centralised.subs([(x_b, xb_inst), (y_b, yb_inst), (z_b, zb_inst), (yaw, yaw_inst), (pitch, pitch_inst), (roll, roll_inst), (q1, q1_inst), (q2, q2_inst), (q3, q3_inst)]).evalf()
sp.pprint(FK_centralised_inst)

State dependencies of centralised kinematics
xe = f({q_2, yaw, roll, x_b, q_1, pitch, q_3})
ye = f({q_2, yaw, roll, pitch, q_1, y_b, q_3})
ze = f({q_2, z_b, roll, q_1, pitch, q_3})
yawe = f({q_2, yaw, roll, q_1, pitch})
pitche = f({q_2, roll, pitch, q_1})
rolle = f({q_2, roll, q_1, pitch, q_3})
Denominators for yaw and roll
-cos(q₂)
sin(q₁)⋅sin(q₃) - cos(q₁)⋅cos(q₂)⋅cos(q₃)
Centralised Jacobian shape: (6, 9)
⎡1.0   0    0   -0.637596694993249  -0.2105          0                  0      ↪
⎢                                                                              ↪
⎢ 0   1.0   0           0              0          0.2105             0.2105    ↪
⎢                                                                              ↪
⎢ 0    0   1.0          0              0     0.637596694993249  0.637596694993 ↪
⎢                                                                              ↪
⎢ 0    0    0          1.0             0             0                  0      ↪
⎢                    